In [1]:
%run "../Common/Enums"

In [2]:
def CopyClientData(lakeName, instance: Enum):
  print("LakeName:", lakeName)
  print("Instance:", instance.name)
  print("InstanceId:", instance.value)
  
  # get last date from the loaded dataset
  LastSyncDate = GetLastSyncDate('company_features', 'client.'+instance.name)
  
  print("Reading data since", LastSyncDate)  
  print("Load CompanyFeatures data")
  PrepareDataBronze (
    lakeName = lakeName,
    instance = instance, 
    entity = BronzeTable.CompanyFeatures,
    entityName = 'tmpCompanyFeatures',
    startDate = LastSyncDate
  )
  
  print("Writing data...")
  
  sourceData = spark.sql("""
    select
          """+str(instance.value)+""" as InstanceId
        , cf.CompanyFeatureId
        , cf.CompanyId
        , cf.FeatureId
        , coalesce(f.FeatureName, 'BAD DATA') as FeatureName
        , cf.CreatedUtc
        , cf.ModifiedUtc
        , cf.Deleted
        , current_timestamp() as SilverModifiedUtc
    from tmpCompanyFeatures cf
      left join tmpFeatures f on f.FeatureId = cf.FeatureId
  """)  
  
  sourceData.registerTempTable("SourceData")
   
  spark.sql("""
    merge into silver.CompanyFeatures as t
    using SourceData as s
      on t.CompanyFeatureId = s.CompanyFeatureId and t.CompanyId = s.CompanyId and t.InstanceId = s.InstanceId 
    when matched then update set 
          FeatureId = s.FeatureId
        , FeatureName = s.FeatureName
        , CreatedUtc = s.CreatedUtc
        , ModifiedUtc = s.ModifiedUtc
        , Deleted = s.Deleted
        , SilverModifiedUtc = s.SilverModifiedUtc
    when not matched then 
      insert ( InstanceId, CompanyId, CompanyFeatureId, FeatureId, FeatureName, CreatedUtc, ModifiedUtc, Deleted, SilverModifiedUtc )
      values ( InstanceId, CompanyId, CompanyFeatureId, FeatureId, FeatureName, CreatedUtc, ModifiedUtc, Deleted, SilverModifiedUtc )
  """);

  print("Writing data completed.")
  
  LastSyncDate = sourceData.agg({"ModifiedUtc": "max"}).collect()[0][0]
#   print(type(LastSyncDate))
  print(LastSyncDate)

  if LastSyncDate is not None:
    UpdateLastSyncDate(LastSyncDate, 'company_features', 'client.'+instance.name)